In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-10-01"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
1542,2024-10-01,França Pro B,15:00,Denain-Voltaire,Boulazac,1.94,1.73,149.5,1.80,1.86,-3.5,1.95,1.72,CnfIvRmS,0.515464,0.578035,0.555556,0.537634,0.093499,216.472,73.442002,0.339268,1.8,1.643168,0.912871,299.92,2.406,0.716331,0.297727,40.0,152.686,64.304245,0.421154,1.2,1.643168,1.369306,93.96,2.158,0.767411,0.355612,-30.0,92,81,3.26,1.16,145.196,179.070,0.080922,0.023184,0.088629,-2.12,-0.424,-2.216981,0.591749,0.4,-0.191749,-0.03,-0.006,-121.666667,0.464106,0.4,-0.064106
1543,2024-10-01,Filipinas Copa Dos Governadores,08:30,NLEX Road Warriors,TNT Tropang Giga,2.60,1.47,193.5,1.87,1.86,4.5,1.92,1.79,QNiEI07D,0.384615,0.680272,0.534759,0.537634,0.064887,308.730,77.321048,0.250449,1.8,1.643168,0.912871,262.99,3.058,0.532466,0.174122,-12.0,140.556,20.488498,0.145768,2.4,1.341641,0.559017,151.51,1.432,0.224544,0.156804,46.0,91,109,2.89,1.39,0.000,0.000,0.392644,0.003791,0.049555,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1544,2024-10-01,França Lnb,15:00,Dijon,Nancy,1.27,3.83,150.5,1.80,1.94,-9.5,1.99,1.76,MoDs0GEd,0.787402,0.261097,0.555556,0.515464,0.048498,171.110,29.614357,0.173072,1.8,1.643168,0.912871,126.15,2.126,0.461118,0.216895,27.0,189.766,123.759152,0.652167,1.8,1.643168,0.912871,93.84,2.342,1.478621,0.631350,-5.0,87,69,1.45,1.36,145.578,472.210,0.709880,0.052938,0.086738,-0.46,-0.092,-2.934783,0.619415,0.5,-0.119415,6.85,1.370,2.065693,0.344187,0.4,0.055813
1545,2024-10-01,França Pro B,15:00,Aix-Maurienne,Fos-sur-Mer,1.56,2.21,161.5,1.80,1.86,-4.5,1.94,1.73,jX59tmJF,0.641026,0.452489,0.555556,0.537634,0.093514,207.930,57.631692,0.277169,1.8,1.643168,0.912871,301.60,2.580,0.799156,0.309750,-10.0,130.942,43.607863,0.333032,2.4,1.341641,0.559017,95.25,1.706,0.581274,0.340724,25.0,80,75,3.77,1.27,154.642,176.052,0.243830,0.023184,0.080922,-1.97,-0.394,-1.421320,0.558168,0.5,-0.058168,-2.01,-0.402,-3.009950,0.446925,0.5,0.053075
1546,2024-10-01,França Pro B,15:00,Caen,Hyeres-Toulon,1.24,3.52,153.5,1.83,1.83,-8.5,2.01,1.68,rVplZ4Bk,0.806452,0.284091,0.546448,0.546448,0.090543,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,65,79,6.98,2.76,0.000,0.000,0.677396,0.000000,0.126474,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1547,2024-10-01,França Pro B,15:00,Nantes,Pau-Orthez,2.04,1.66,151.5,1.83,1.83,-2.5,1.95,1.72,ImaWoree,0.490196,0.602410,0.546448,0.546448,0.092606,252.316,113.838596,0.451175,0.6,1.341641,2.236068,323.25,3.216,1.575113,0.489774,-52.0,155.888,42.075167,0.269906,1.8,1.643168,0.912871,123.28,2.116,0.649407,0.306903,-12.0,75,92,4.31,1.34,130.104,205.848,0.145244,0.000000,0.088629,-2.12,-0.424,-2.452830,0.706541,0.6,-0.106541,-0.73,-0.146,-4.520548,0.397279,0.3,-0.097279
1548,2024-10-01,Europa Liga Da Aba 2,15:00,Ilirija,Borac Banja Luka,1.95,1.72,157.5,1.85,1.81,1.5,1.83,1.83,dMb9WgFc,0.512821,0.581395,0.540541,0.552486,0.094216,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,0,0.00,0.00,0.000,0.000,0.088629,0.015456,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1549,2024-10-01,França Pro B

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
1561,14:00,Europa Eurocopa,Ulm,Joventut,2.45,Back Home
1564,20:30,Eua Wnba,New York Liberty F,Las Vegas Aces F,1.61,Back Home
